# Feature Engineering Script

**After Second Bus (SB) Data, waze data by the same street and default waze data**

Input variables: 89 + 1 (id)

In [1]:
# importing required packages
import pandas as pd
# import pandas_profiling
import numpy as np
import os
import featuretools as ft

# To install packages, run on cmd:
# pip3 install pandas-profiling
# pip3 install featuretools

In [2]:
# Choose the city of analysis

city = 'Recife'
# city = 'CG'
# city = 'Curitiba'

In [3]:
# Put the integrated data path

integrated_data_path = os.getcwd() + "/../data/output/" + city + "/"

In [4]:
# Importing the data

output_files = []
days_of_analysis = 0

for dir_name in os.listdir(integrated_data_path):
    
    if dir_name.startswith("output_"): #to get the dir per day
        dir_path = integrated_data_path + dir_name
        days_of_analysis = days_of_analysis+1
        
        for file_name in os.listdir(dir_path):
            
            if file_name.startswith("part-"):
                file_path = dir_path + "/" + file_name
                
                # Join multiple files of the same day
                df = pd.read_csv(file_path, index_col=None, header=0)
                output_files.append(df)
                
print('Days of analysis: ', days_of_analysis)

# Join all output files            
all_integrated_data = pd.concat(output_files, axis=0, ignore_index=True)

all_integrated_data.head(5)

/home/veruska/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (37,82) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/veruska/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (82) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Days of analysis:  12


,route,tripNum,shapeId,routeFrequency,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,...,alertIsJamUnifiedAlertSB,alertInScaleSB,jamUpdateDateTimeSB,jamExpirationDateTimeSB,jamBlockTypeSB,jamDelaySB,jamLengthSB,jamLevelSB,jamSeveritySB,jamSpeedKMSB
0,74,1,Route_174,low_frequency,13735,-8.020711,-34.923093,4061.0,50474,NaN,...,False,False,2018-12-06 04:55:11,NaN,NORMAL,0,0,0,0,NaN
1,74,1,Route_174,low_frequency,13735,-8.020711,-34.923093,4061.0,50477,NaN,...,False,False,2018-12-06 05:23:37,NaN,NORMAL,0,0,0,0,NaN
2,74,1,Route_174,low_frequency,13735,-8.020711,-34.923093,4061.0,50212,NaN,...,False,False,2018-12-06 05:36:36,NaN,NORMAL,0,0,0,0,NaN
3,74,1,Route_174,low_frequency,13735,-8.020711,-34.923093,4061.0,50242,NaN,...,False,False,2018-12-06 05:43:41,NaN,NORMAL,0,0,0,0,NaN
4,74,1,Route_174,low_frequency,13735,-8.020711,-34.923093,4061.0,50463,NaN,...,False,False,2018-12-06 05:49:03,NaN,NORMAL,0,0,0,0,NaN


In [5]:
# Sorting by bus1-bus2-time

all_integrated_data.sort_values(by=['busCode', "busCodeSB", "gps_datetime"], inplace=True)
all_integrated_data.head(5)

,route,tripNum,shapeId,routeFrequency,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,...,alertIsJamUnifiedAlertSB,alertInScaleSB,jamUpdateDateTimeSB,jamExpirationDateTimeSB,jamBlockTypeSB,jamDelaySB,jamLengthSB,jamLevelSB,jamSeveritySB,jamSpeedKMSB
1235794,68,3,Route_160,low_frequency,12580,-8.036231,-34.962342,2505.0,12021,NaN,...,False,False,2018-12-03 07:47:15,NaN,NaN,338,719,4,1,6.00
1128197,68,3,Route_160,low_frequency,12581,-8.034150,-34.960756,2794.0,12021,130.0,...,False,False,2018-12-03 07:37:00,NaN,NORMAL,0,0,0,0,NaN
1190868,68,3,Route_160,low_frequency,12582,-8.032878,-34.959775,2971.0,12021,NaN,...,False,False,2018-12-03 07:47:15,NaN,NaN,338,719,4,1,6.00
1232471,68,3,Route_160,low_frequency,12583,-8.031478,-34.958622,3171.0,12021,131.0,...,False,False,2018-12-03 08:27:38,NaN,NaN,878,618,4,1,2.32
1233796,68,3,Route_160,low_frequency,12585,-8.030508,-34.957567,3392.0,12021,136.0,...,False,False,2018-12-03 08:07:58,NaN,NaN,1568,3199,4,2,6.24


The following variables are objetc type and should be converted to the origin types because the correlation matrix.

In [6]:
all_integrated_data.dtypes

route               int64
tripNum             int64
shapeId            object
routeFrequency     object
shapeSequence       int64
                   ...   
jamDelaySB          int64
jamLengthSB         int64
jamLevelSB          int64
jamSeveritySB       int64
jamSpeedKMSB      float64
Length: 89, dtype: object

In [6]:
all_integrated_data['shapeId'] = all_integrated_data['shapeId'].astype(str)
all_integrated_data['routeFrequency'] = all_integrated_data['routeFrequency'].astype(str)
all_integrated_data['gps_datetime'] = all_integrated_data['gps_datetime'].astype('datetime64[ns]')
all_integrated_data['problem'] = all_integrated_data['problem'].astype(str)
all_integrated_data['precipitationTime'] = all_integrated_data['precipitationTime'].astype('datetime64[ns]')
all_integrated_data['alertDateTime'] = all_integrated_data['alertDateTime'].astype('datetime64[ns]')
all_integrated_data['alertSubtype'] = all_integrated_data['alertSubtype'].astype(str)
all_integrated_data['alertSubtype'].replace("nan", "-", inplace=True)
all_integrated_data['alertType'] = all_integrated_data['alertType'].astype(str)
all_integrated_data['alertType'].replace("nan", "-", inplace=True)
all_integrated_data['alertIsJamUnifiedAlert'] = all_integrated_data['alertIsJamUnifiedAlert'].astype(bool)
all_integrated_data['alertInScale'] = all_integrated_data['alertInScale'].astype(bool)
all_integrated_data['jamUpdateDateTime'] = all_integrated_data['jamUpdateDateTime'].astype('datetime64[ns]')
all_integrated_data['jamExpirationDateTime'] = all_integrated_data['jamExpirationDateTime'].astype('datetime64[ns]')
all_integrated_data['jamBlockType'] = all_integrated_data['jamBlockType'].astype(str)
all_integrated_data['jamBlockType'].replace("nan", "-", inplace=True)

all_integrated_data['gps_datetimeSB'] = all_integrated_data['gps_datetimeSB'].astype('datetime64[ns]')
all_integrated_data['problemSB'] = all_integrated_data['problemSB'].astype(str)
all_integrated_data['precipitationTimeSB'] = all_integrated_data['precipitationTimeSB'].astype('datetime64[ns]')
all_integrated_data['alertDateTimeSB'] = all_integrated_data['alertDateTimeSB'].astype('datetime64[ns]')
all_integrated_data['alertSubtypeSB'] = all_integrated_data['alertSubtypeSB'].astype(str)
all_integrated_data['alertSubtypeSB'].replace("nan", "-", inplace=True)
all_integrated_data['alertTypeSB'] = all_integrated_data['alertTypeSB'].astype(str)
all_integrated_data['alertTypeSB'].replace("nan", "-", inplace=True)
all_integrated_data['alertIsJamUnifiedAlertSB'] = all_integrated_data['alertIsJamUnifiedAlertSB'].astype(bool)
all_integrated_data['alertInScaleSB'] = all_integrated_data['alertInScaleSB'].astype(bool)
all_integrated_data['jamUpdateDateTimeSB'] = all_integrated_data['jamUpdateDateTimeSB'].astype('datetime64[ns]')
all_integrated_data['jamExpirationDateTimeSB'] = all_integrated_data['jamExpirationDateTimeSB'].astype('datetime64[ns]')
all_integrated_data['jamBlockTypeSB'] = all_integrated_data['jamBlockTypeSB'].astype(str)
all_integrated_data['jamBlockTypeSB'].replace("nan", "-", inplace=True)

all_integrated_data.dtypes

route               int64
tripNum             int64
shapeId            object
routeFrequency     object
shapeSequence       int64
                   ...   
jamDelaySB          int64
jamLengthSB         int64
jamLevelSB          int64
jamSeveritySB       int64
jamSpeedKMSB      float64
Length: 89, dtype: object

In [8]:
# Descriptive statistics
profile = pandas_profiling.ProfileReport(all_integrated_data, check_correlation=True, correlation_threshold=0.5)

In [9]:
profile.to_file(outputfile=integrated_data_path + "output_profile_integrated_data.html")

In [7]:
# deleting jamExpirationDateTime because there is no value
all_integrated_data.drop('jamExpirationDateTime', axis=1, inplace=True)
all_integrated_data.drop('jamExpirationDateTimeSB', axis=1, inplace=True)

In [8]:
# saving integrated data before generate new feats
output_path = integrated_data_path + "/" + "integrated_data_before_new_feats.csv"
all_integrated_data.to_csv(output_path, index=False)

* Maybe it didn't measuring correlation because a lot of variables.

### 2. Creating new features


#### 2.1 With Feature Tools

In [9]:
# Creating an entity set
es = ft.EntitySet(id="all_data")

entities = es.entity_from_dataframe(entity_id="id", dataframe=all_integrated_data, make_index=True, index="id", time_index='gps_datetime')
entities

Entityset: all_data
  Entities:
    id [Rows: 1432633, Columns: 88]
  Relationships:
    No relationships

In [10]:
entities.entities

[Entity: id
   Variables:
     id (dtype: index)
     route (dtype: numeric)
     tripNum (dtype: numeric)
     shapeId (dtype: categorical)
     routeFrequency (dtype: categorical)
     shapeSequence (dtype: numeric)
     shapeLat (dtype: numeric)
     shapeLon (dtype: numeric)
     distanceTraveledShape (dtype: numeric)
     busCode (dtype: numeric)
     gpsPointId (dtype: numeric)
     gpsLat (dtype: numeric)
     gpsLon (dtype: numeric)
     distanceToShapePoint (dtype: numeric)
     gps_datetime (dtype: datetime_time_index)
     stopPointId (dtype: numeric)
     problem (dtype: categorical)
     precipitation (dtype: numeric)
     precipitationTime (dtype: datetime)
     precipitationStationDistance (dtype: numeric)
     alertDateTime (dtype: datetime)
     alertSubtype (dtype: categorical)
     alertType (dtype: categorical)
     alertRoadType (dtype: numeric)
     alertConfidence (dtype: numeric)
     alertNComments (dtype: numeric)
     alertNImages (dtype: numeric)
     alertN

**Para a variável alvo (dados de ônibus por parada de ônibus), podemos gerar as seguintes variáveis:**

In [12]:
feature_matrix, features_defs = ft.dfs(entityset=entities, target_entity="id")

In [13]:
# Saving file
output_path = integrated_data_path + "/" + "new_feats_data.csv"
feature_matrix.to_csv(output_path, index=False)

feature_matrix.head(5)

,route,tripNum,shapeId,routeFrequency,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,...,MONTH(alertDateTimeSB),MONTH(jamUpdateDateTimeSB),WEEKDAY(gps_datetime),WEEKDAY(precipitationTime),WEEKDAY(alertDateTime),WEEKDAY(jamUpdateDateTime),WEEKDAY(gps_datetimeSB),WEEKDAY(precipitationTimeSB),WEEKDAY(alertDateTimeSB),WEEKDAY(jamUpdateDateTimeSB)
id,,,,,,,,,,,,,,,,,,,,,
0,68,3,Route_160,low_frequency,12580,-8.036231,-34.962342,2505.0,12021,NaN,...,12,12,0,0,0,0,0,0,0,0
1,68,3,Route_160,low_frequency,12581,-8.034150,-34.960756,2794.0,12021,130.0,...,12,12,0,0,0,0,0,0,0,0
2,68,3,Route_160,low_frequency,12582,-8.032878,-34.959775,2971.0,12021,NaN,...,12,12,0,0,0,0,0,0,0,0
3,68,3,Route_160,low_frequency,12583,-8.031478,-34.958622,3171.0,12021,131.0,...,12,12,0,0,0,0,0,0,0,0
4,68,3,Route_160,low_frequency,12585,-8.030508,-34.957567,3392.0,12021,136.0,...,12,12,0,0,0,0,0,0,0,0


In [14]:
features_defs

[<Feature: route>,
 <Feature: tripNum>,
 <Feature: shapeId>,
 <Feature: routeFrequency>,
 <Feature: shapeSequence>,
 <Feature: shapeLat>,
 <Feature: shapeLon>,
 <Feature: distanceTraveledShape>,
 <Feature: busCode>,
 <Feature: gpsPointId>,
 <Feature: gpsLat>,
 <Feature: gpsLon>,
 <Feature: distanceToShapePoint>,
 <Feature: stopPointId>,
 <Feature: problem>,
 <Feature: precipitation>,
 <Feature: precipitationStationDistance>,
 <Feature: alertSubtype>,
 <Feature: alertType>,
 <Feature: alertRoadType>,
 <Feature: alertConfidence>,
 <Feature: alertNComments>,
 <Feature: alertNImages>,
 <Feature: alertNThumbsUp>,
 <Feature: alertReliability>,
 <Feature: alertReportMood>,
 <Feature: alertReportRating>,
 <Feature: alertSpeed>,
 <Feature: alertLatitude>,
 <Feature: alertLongitude>,
 <Feature: alertDistanceToClosestShapePoint>,
 <Feature: alertIsJamUnifiedAlert>,
 <Feature: alertInScale>,
 <Feature: jamBlockType>,
 <Feature: jamDelay>,
 <Feature: jamLength>,
 <Feature: jamLevel>,
 <Feature: jam

About feature types:
* It works with data type, but generate just mean, sum, count, mode...